# Get the connector to dremio with the class DremioQueryMakerJDBC

In [1]:
%run /home/ubuntu/notebooks/covid-prj01-data-analysis/[UTILS]/dremio-access-new.py
dremio = DremioQueryMakerJDBC()
index = dremio.getIndex()
display(index)

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,VIEW_DEFINITION
0,DREMIO,covidlake-raw,brasil_io,"SELECT * FROM ""brasil.io"""
1,DREMIO,covidlake-raw,google_trends,"SELECT * FROM ""google_trends.parquet"""
2,DREMIO,covidlake-raw,health_ministry,SELECT * FROM health_ministry
3,DREMIO,covidlake-raw,hospitals_dimension,"SELECT * FROM ""hospitals.csv"""
4,DREMIO,covidlake-raw,wcota,SELECT * FROM wcota
5,DREMIO,PowerBI,VDS-cases-state,"select a.state,a.city, sum(a.confirmed) as c..."
6,DREMIO,covidlake-raw.ipea,dolar_ratio,"SELECT * FROM ""dolar_ratio.parquet"""


# Get tables from S3

In [44]:
df = pd.read_csv("s3://covid-19-prj/raw/hospitals/hospitals.csv", sep = ";")
df["UF"] = df["UF,HOSPITAL"].apply(lambda x: x.split(",")[0])
df["NOME"] = df["UF,HOSPITAL"].apply(lambda x: ",".join(x.split(",")[1:]))
df.drop(columns = ["UF,HOSPITAL"], inplace = True)
df.head()

,UF,NOME
0,AC,Hospital Geral de Clínicas de Rio Branco
1,AL,HOSPITAL ESCOLA DR. HÉLVIO AUTO
2,AM,Instituto da Criança do Amazonas
3,AM,Instituto da Mulher Dona Lindu
4,AM,Hospital Infantil Dr. Fajardo


In [46]:
from geopy.geocoders import Nominatim

In [47]:
geolocator = Nominatim(user_agent="covid-lake")
locations = [geolocator.geocode(loc + ", " + state + ", " + "BRASIL") 
             for loc, state in zip(df["NOME"],df["UF"])]
locations[:5]

[None,
 None,
 Location(Instituto da Criança do Amazonas, Avenida Codajás, Cachoeirinha, Manaus, Microrregião de Manaus, Mesorregião Centro Amazonense, Amazonas, Região Norte, 69000-000, Brasil, (-3.1195401, -60.0040424, 0.0)),
 None,
 None]

In [48]:
df["RUA"] = [loc[0].split(",")[1][1:] if loc is not None else "" for loc in locations]
df["BAIRRO"] = [loc[0].split(",")[2][1:] if loc is not None else "" for loc in locations]
df["CIDADE"] = [loc[0].split(",")[3][1:] if loc is not None else "" for loc in locations]
df["MICROREGIAO"] = [loc[0].split(",")[4][1:] if loc is not None else "" for loc in locations]
df["MESOREGIAO"] = [loc[0].split(",")[5][1:] if loc is not None else "" for loc in locations]
df["ESTADO"] = [loc[0].split(",")[6][1:] if loc is not None else "" for loc in locations]
df["CEP"] = [loc[0].split(",")[7][1:] if loc is not None else "" for loc in locations]
df["LAT"] = [loc[1][0] if loc is not None else "" for loc in locations]
df["LON"] = [loc[1][1] if loc is not None else "" for loc in locations]

In [50]:
df.to_csv("s3://covid-19-prj/raw/hospitals/hospitals_loc.csv", sep = ";")